<a href="https://colab.research.google.com/github/GGMmattos/TCC-Informatica/blob/main/C%C3%B3digo_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libs

In [ ]:

#libs
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import load_dataset
import numpy as np
import torch
from datasets import Dataset
import re
import time
from transformers import AutoTokenizer, AutoModel
from google.colab import files
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


!pip install pytorch-lightning -q
!pip install coral_pytorch -q
!pip install transformers==4.30.0 -q # Install a specific version of transformers

import torch
import os
import json # Para lidar com os embeddings salvos como string JSON
import ast # Para avaliar strings de lista de notas
import shlex # Para o NILC Metrix (se ainda precisar rodar o script no mesmo arquivo)

# Para PyTorch Lightning
import pytorch_lightning as pl
import torchmetrics
from coral_pytorch.layers import CoralLayer
from coral_pytorch.losses import coral_loss
from coral_pytorch.dataset import levels_from_labelbatch, proba_to_label

# Para Sklearn (pré-processamento e divisão de dados)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from torch.utils.data import Dataset, DataLoader

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger

# Função de calculo acurácia do ENEM

In [ ]:
import torch
from torchmetrics import Metric
from typing import Any

class ENEMAccuracy(Metric):
    def __init__(self, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)

        # O limite de 80 pontos na escala original do ENEM
        # corresponde a um limite de 2 na sua escala ordinal (0 a 5).
        # A diferença absoluta máxima para ser 'não-divergente' é 2.
        self.divergence_threshold = 2
        self.add_state("non_divergent_count", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("total_count", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        # A `preds` é o tensor de rótulos previstos pelo modelo.
        # A `target` é o tensor de rótulos verdadeiros.

        # Calcula a diferença absoluta entre a previsão e o valor real
        diff = torch.abs(preds - target)

        # Conta as previsões que estão dentro do limite de divergência
        non_divergent_batch = torch.sum(diff <= self.divergence_threshold)

        self.non_divergent_count += non_divergent_batch
        self.total_count += target.numel()

    def compute(self):
        # Calcula a precisão final como a porcentagem de previsões não-divergentes
        return self.non_divergent_count.float() / self.total_count.float()

# Importação do dataset

**2018-Fonseca et al**

* Automatically Grading Brazilian Student Essays - NILC Metrix  - [link NILC](http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html) -  https://github.com/nilc-nlp/nilcmetrix



Importação do dataset [Link](https://huggingface.co/datasets/kamel-usp/aes_enem_dataset)

In [ ]:
# # Login using e.g. `huggingface-cli login` to access this dataset
# splits = {'train': 'PROPOR2024/train-00000-of-00001.parquet', 'validation': 'PROPOR2024/validation-00000-of-00001.parquet', 'test': 'PROPOR2024/test-00000-of-00001.parquet'}
# df_train = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["train"])

In [ ]:
#Função de ordenação
# def natural_sort_key(s):
#     """
#     Função para ordenação natural de strings que contêm números.
#     Exemplo: ['ID10', 'ID2'] será ordenado como ['ID2', 'ID10'] em vez de ['ID10', 'ID2']
#     """
#     return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# # Ordenar o DataFrame usando a chave de ordenação natural
# df_test = df_test.iloc[pd.Index(df_test['id_texto']).map(lambda x: natural_sort_key(x)).argsort()]
# df_test.reset_index(drop=True, inplace=True)


In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics/df_train.csv", encoding='latin-1')
# df_validation = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics/df_validation.csv", encoding='latin-1')
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics/df_test.csv", encoding='latin-1')

In [ ]:
import pandas as pd

In [ ]:
df_validation = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/textos + NILC/df_val.csv", encoding='latin-1',     decimal=',')


In [ ]:
df_validation['honore']

In [ ]:
# df_train.shape

In [ ]:
# df_validation.shape

In [ ]:
# df_test.shape

In [ ]:
# # Concatenando verticalmente
# df = pd.concat([df_train, df_validation, df_test], ignore_index=True)

In [ ]:
# df.drop(columns='id_texto', inplace=True)

In [ ]:
# df.shape

In [ ]:
# df.to_csv('df.csv', index=False)

# Gerando embedding


In [ ]:
# # Importa as classes necessárias da biblioteca 'transformers'
# from transformers import AutoTokenizer, AutoModel
# import torch

# # --- 1. Definição do Tokenizer e do Modelo ---
# # O AutoTokenizer e o AutoModel se encarregam de carregar as configurações
# # corretas para o modelo BERTimbau.
# tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
# model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

# # --- 2. Movendo para a GPU (prática recomendada) ---
# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     model.to(device)
#     print("Modelo BERTimbau carregado e movido para a GPU.")
# else:
#     device = torch.device("cpu")
#     print("Modelo BERTimbau carregado e usando a CPU.")

In [ ]:
import pandas as pd

In [ ]:
# Importação dos dataset com text + metrix
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/textos + NILC/df_test.csv", encoding='latin-1', decimal=',')
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/textos + NILC/df_train.csv", encoding='latin-1', decimal=',')
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/textos + NILC/df_val.csv", encoding='latin-1', decimal=',')


In [ ]:
df_test.shape

In [ ]:
df_train.shape

In [ ]:
df_val.shape

BERTimbau (como a maioria dos BERTs base) tem um limite de 512 tokens para a entrada. Redações do ENEM geralmente são mais longas.

**Código para obter embedding (segmentação)**

Dividir a redação em pedaços de 512 tokens, gerar um embedding para cada pedaço e depois combiná-los

In [ ]:
def get_bert_embedding_segmented(text, tokenizer, model, device, max_length=512, stride=256):
    """
    Gera um embedding BERT para um texto longo, utilizando segmentação e agregação.

    Args:
        text (str): O texto da redação.
        tokenizer: O tokenizer do modelo BERT.
        model: O modelo BERT (ex: BERTimbau).
        device (torch.device): 'cuda' ou 'cpu'.
        max_length (int): O tamanho máximo de cada segmento para o BERT (default: 512).
        stride (int): O tamanho da sobreposição entre os segmentos (default: 256).
                      Um stride menor significa mais sobreposição e possivelmente
                      maior captura de contexto, mas mais processamento.

    Returns:
        np.array: O embedding combinado (média) de todos os segmentos, ou NaNs se o texto for nulo.
    """
    if pd.isna(text) or not isinstance(text, str): # Lidar com possíveis valores NaN ou não-string
        return np.full((model.config.hidden_size,), np.nan) # Retorna um array de NaNs

    # Tokenizar o texto completo com o retorno de IDs (para controlar a segmentação)
    # add_special_tokens=False para não adicionar [CLS]/[SEP] no começo e fim de cada segmento
    # pois queremos controlar isso.
    token_ids = tokenizer.encode(text, add_special_tokens=False)

    # Lista para armazenar os embeddings de cada segmento
    segment_embeddings = []

    # Iterar sobre os tokens para criar segmentos
    # max_length - 2 para dar espaço para [CLS] e [SEP] que serão adicionados pelo tokenizer em cada segmento
    effective_max_length = max_length - 2

    # Se o texto for muito curto, trata como um único segmento (mesmo que menor que max_length)
    if len(token_ids) <= effective_max_length:
        segments = [token_ids]
    else:
        segments = []
        for i in range(0, len(token_ids), effective_max_length - stride):
            segment = token_ids[i : i + effective_max_length]
            segments.append(segment)
            # Se o último segmento for menor que o stride (não suficiente para sobreposição),
            # ou se já chegamos ao final, paramos.
            if i + effective_max_length >= len(token_ids):
                break


    # Processar cada segmento
    for segment in segments:
        # Adicionar os tokens especiais para cada segmento individualmente
        # return_tensors='pt' para PyTorch
        # truncation=True é redundante aqui se já controlamos o tamanho, mas é bom manter
        # padding='max_length' garante que todos os segmentos tenham o mesmo tamanho de input
        inputs = tokenizer.prepare_for_model(
            segment,
            add_special_tokens=True,
            return_tensors='pt',
            max_length=max_length,
            padding='max_length',
            truncation=True # Em caso de erro na lógica de segmentação, isso garante
        )

        # Add batch dimension to each tensor in the inputs dictionary
        inputs = {k: v.unsqueeze(0).to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        # Retorna o embedding do token [CLS] do segmento
        # outputs.last_hidden_state will now have shape (batch_size, sequence_length, hidden_size)
        # We still want the CLS token for the single item in the batch, which is index 0
        segment_embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()
        segment_embeddings.append(segment_embedding)

    if not segment_embeddings: # Se por algum motivo nenhum embedding foi gerado (ex: texto vazio após pré-processamento)
        return np.full((model.config.hidden_size,), np.nan)

    # Combinar os embeddings dos segmentos pela média
    combined_embedding = np.mean(segment_embeddings, axis=0)

    return combined_embedding

In [ ]:
# 1. Carregar o Tokenizer e o Modelo BERTimbau
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Mover o modelo para a GPU, se disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval() # Coloca o modelo em modo de avaliação

In [ ]:
df_val['bert_embedding'] = df_val['essay_text'].apply(
    lambda text: get_bert_embedding_segmented(text, tokenizer, model, device, max_length=512, stride=256)
)

print("\nDataFrame com embeddings BERTimbau segmentados:")
print(df_val.head())

#Para verificar o formato do primeiro embedding:
print(f"\nShape do primeiro embedding segmentado: {df_val['bert_embedding'].iloc[0].shape}")

In [ ]:
nome_do_arquivo_csv = 'df_val.csv'
df_val.to_csv(nome_do_arquivo_csv, index=False)

# Ajuste para regressão ordinal

In [ ]:
# Importação dos dataset com text + metrix + embeddings
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding/df_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding/df_test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding/df_val.csv")

Processamento da coluna de notas


In [ ]:

# 1. Converter as strings de array para listas/arrays NumPy reais
print("Convertendo strings de notas (coluna 'grades') para listas de números...")
df_val['grades_parsed'] = df_val['grades'].apply(
    # 1. .strip('[]') remove os colchetes
    # 2. .split() divide por qualquer espaço em branco (um ou vários)
    # 3. int(s) converte cada parte para inteiro
    lambda x: [int(s) for s in x.strip('[]').split()] if isinstance(x, str) else x
)
print(f"Exemplo da coluna 'grades_parsed':\n{df_val['grades_parsed'].head()}")


In [ ]:
# 2. Extrair cada nota de competência e a nota total
print("Extraindo notas para cada competência e a nota total...")
# Nomes das colunas para as notas de competência
competencia_cols = [f'nota_competencia_{i+1}' for i in range(5)]
# Coluna para a nota total
nota_total_col = 'nota_final_redacao'

# Iterar para criar as colunas de competência e a nota final
for i in range(6): # Iterar de 0 a 5 para pegar os 6 valores do vetor
    col_name = ''
    if i < 5:
        col_name = competencia_cols[i]
    else: # O sexto elemento (índice 5) é a nota total
        col_name = nota_total_col

    df_val[col_name] = df_val['grades_parsed'].apply(
        # Verifica se é uma lista e tem o tamanho esperado (6 elementos)
        lambda x: x[i] if isinstance(x, list) and len(x) == 6 else np.nan
    )
    # Garante que as notas sejam inteiras
    df_val[col_name] = df_val[col_name].astype(int)

print("Exemplo das novas colunas de notas de competência e total:")
print(df_val[competencia_cols + [nota_total_col]].head())
print("\n---")

In [ ]:
df_train.shape

In [ ]:
# 4. Definir NUM_CLASSES para cada Competência e Mapeamento Ordinal
NUM_CLASSES_COMPETENCIA = 6 # 0, 40, 80, 120, 160, 200

# Mapeamento para o formato ordinal (0 a 5)
notas_competencia_unicas = np.array([0, 40, 80, 120, 160, 200])
mapeamento_competencia_ordinal = {nota: i for i, nota in enumerate(notas_competencia_unicas)}

# Aplicar o mapeamento para cada coluna de competência
print(f"Mapeando notas de competência para o formato ordinal (0 a {NUM_CLASSES_COMPETENCIA-1})")
for col_name in competencia_cols:
    df_val[f'{col_name}_ordinal'] = df_val[col_name].map(mapeamento_competencia_ordinal)

print("Exemplo das notas ordinais das competências:")
print(df_val[[f'{col_name}_ordinal' for col_name in competencia_cols]].head())

In [ ]:
nome_do_arquivo_csv = 'df_val.csv'
df_val.to_csv(nome_do_arquivo_csv, index=False)

# Configurações do modelo

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Dataframe final/DataFrame_Final-2.0.csv')

Configurações gerais e hiperparâmetros


In [ ]:
BATCH_SIZE = 128
NUM_EPOCHS = 400
LEARNING_RATE = 0.01
NUM_WORKERS = 0 # Este parâmemtro pode ajudar na velocidade do treinamento não na qualidade do modelo em termos das métricas
# Notas de competência do ENEM: 0, 40, 80, 120, 160, 200
NUM_CLASSES_COMPETENCIA = 6

**MultiLayerPerceptron**: Esta é a rede neural em "PyTorch puro". Ela define a arquitetura do Perceptron Multicamadas e incorpora a camada de saída especial do CORAL (CoralLayer).

In [ ]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, input_size, hidden_units, num_classes):
        super().__init__() # Chama o construtor da classe base torch.nn.Module

        # Armazena o número de classes, necessário para a função de perda CORAL
        self.num_classes = num_classes

        # Lista para armazenar todas as camadas da MLP
        all_layers = []

        # Loop para criar as camadas ocultas da MLP
        # 'hidden_units' é uma tupla ou lista (ex: (256, 128, 64))
        # Cada 'hidden_unit' representa o número de neurônios em uma camada oculta.
        for hidden_unit in hidden_units:
            # Cria uma camada linear (totalmente conectada)
            # 'input_size' é o número de entradas para esta camada (saída da camada anterior ou features iniciais)
            # 'hidden_unit' é o número de saídas desta camada
            layer = torch.nn.Linear(input_size, hidden_unit)
            all_layers.append(layer)

            # Adiciona uma função de ativação ReLU após cada camada linear (exceto a última do CORAL)
            all_layers.append(torch.nn.ReLU())

            # Atualiza o 'input_size' para a próxima camada ser a saída da camada atual
            input_size = hidden_unit

        # --- Camada de Saída CORAL ---
        # Esta é a principal adaptação para Regressão Ordinal com CORAL.
        # Ao invés de uma camada linear normal (torch.nn.Linear) que retornaria um vetor de logits para classificação multiclasse,
        # usamos a CoralLayer da biblioteca coral_pytorch.
        # `size_in`: número de entradas para esta camada (que é a saída da última camada oculta, hidden_units[-1])
        # `num_classes`: o número total de categorias de pontuação (ex: 6 para 0 a 200).
        output_layer = CoralLayer(size_in=hidden_units[-1], num_classes=num_classes)
        all_layers.append(output_layer)

        # Combina todas as camadas em um modelo sequencial.
        # 'Sequential' executa as camadas em ordem, uma após a outra.
        self.model = torch.nn.Sequential(*all_layers)

    def forward(self, x):
        """
        Define o fluxo de dados para frente através da rede neural.
        `x` é o tensor de entrada (as features).
        """
        # Passa o tensor de entrada através da sequência de camadas definidas
        x = self.model(x)
        return x

**LightningMLP**: Esta é uma classe do PyTorch Lightning. Ela "envolve" a rede neural (MultiLayerPerceptron) e adiciona toda a funcionalidade extra que o Lightning oferece (treinamento automatizado, validação, teste, logs, otimizadores, etc.), tornando o código de treino muito mais limpo e padronizado.

In [ ]:
class LightningMLP(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__() # Chama o construtor da classe base pl.LightningModule

        self.learning_rate = learning_rate # Taxa de aprendizado para o otimizador
        self.model = model # A instância da rede neural MultiLayerPerceptron

        # Salva configurações e hiperparâmetros (como learning_rate) no diretório de log.
        # 'ignore=['model']' impede que os parâmetros do modelo sejam salvos duas vezes.
        self.save_hyperparameters(ignore=['model'])

        # MODIFICAÇÃO REALIZADA AQUI
        # Inicializa a métrica ENEMAccuracy para cada etapa
        self.train_enem_acc = ENEMAccuracy()
        self.valid_enem_acc = ENEMAccuracy()
        self.test_enem_acc = ENEMAccuracy()

        self.train_rmse = torchmetrics.MeanSquaredError(squared=False)
        self.valid_rmse = torchmetrics.MeanSquaredError(squared=False)
        self.test_rmse = torchmetrics.MeanSquaredError(squared=False)


        # --- Configuração das Métricas de Avaliação ---
        # pythorthmetrics são uma forma conveniente de calcular métricas.
        # Mean Absolute Error (MAE): Mede a diferença média absoluta entre previsões e rótulos verdadeiros.
        self.train_mae = torchmetrics.MeanAbsoluteError()
        self.valid_mae = torchmetrics.MeanAbsoluteError()
        self.test_mae = torchmetrics.MeanAbsoluteError()

        # Quadratic Weighted Kappa (QWK): Métrica CRUCIAL para AES.
        # Ela considera a natureza ordinal das notas e penaliza erros maiores mais severamente.
        # `num_classes`: número de categorias de nota (ex: 6 para competências).
        # `task='multiclass'`: indica que é uma tarefa de classificação multi-classe (embora seja ordinal).
        # `weights='quadratic'`: Aplica os pesos quadráticos para penalizar mais erros maiores.
        self.train_qwk = torchmetrics.CohenKappa(num_classes=self.model.num_classes, task='multiclass', weights='quadratic')
        self.valid_qwk = torchmetrics.CohenKappa(num_classes=self.model.num_classes, task='multiclass', weights='quadratic')
        self.test_qwk = torchmetrics.CohenKappa(num_classes=self.model.num_classes, task='multiclass', weights='quadratic')

    def forward(self, x):
        """
        Define o fluxo de dados para frente através do modelo PyTorch.
        Este método é chamado internamente pelo Lightning para fazer previsões.
        """
        return self.model(x) # Simplesmente passa a entrada para o MultiLayerPerceptron

    def _shared_step(self, batch):
        """
        Etapa comum de processamento que é usada para treinamento, validação e teste.
        Isso evita a duplicação de código.
        """
        features, true_labels = batch # Desempacota o batch de dados

        # --- Adaptação CORAL: Converter labels para o formato binário estendido ---
        # 'levels_from_labelbatch' é uma função de coral_pytorch.
        # Ela transforma um rótulo de classe inteira (ex: 3) em um vetor binário de "níveis" (ex: [1, 1, 1, 0, 0, 0])
        # `num_classes`: o número total de classes.
        levels = levels_from_labelbatch(
            true_labels, num_classes=self.model.num_classes)

        # Passa as features para a rede neural para obter os logits (saídas brutas)
        logits = self(features) # self(features) é o mesmo que self.forward(features)

        # --- Adaptação CORAL: Calcular a função de perda CORAL ---
        # 'coral_loss' é uma função de coral_pytorch.
        # Ela calcula a perda entre os logits do modelo e os 'levels' binários.
        # 'levels.type_as(logits)' garante que os tipos de dados dos tensores sejam compatíveis.
        loss = coral_loss(logits, levels.type_as(logits))

        # --- Adaptação CORAL: Converter probabilidades previstas em rótulos finais ---
        # `torch.sigmoid(logits)`: Transforma os logits brutos em probabilidades entre 0 e 1.
        # `proba_to_label`: Uma função de coral_pytorch que converte essas probabilidades
        # em rótulos de classe previstos (ex: 0, 1, 2, ..., num_classes-1).
        probas = torch.sigmoid(logits)
        predicted_labels = proba_to_label(probas)

        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        """
        Define o que acontece em cada passo de treinamento (para cada batch).
        """
        loss, true_labels, predicted_labels = self._shared_step(batch) # Usa a etapa compartilhada

        # MODIFICADO AQUI
        self.train_enem_acc(predicted_labels.float(), true_labels.float())
        self.log("train_enem_acc", self.train_enem_acc, on_epoch=True, on_step=False)

        # Registrar a perda de treinamento
        self.log("train_loss", loss, on_epoch=True, on_step=False) # 'on_epoch=True' loga no final da epoch

        # Calcular e registrar o MAE de treinamento
        self.train_mae(predicted_labels, true_labels)
        self.log("train_mae", self.train_mae, on_epoch=True, on_step=False)

        # Calcular e registrar o QWK de treinamento
        self.train_qwk(predicted_labels, true_labels)
        self.log("train_qwk", self.train_qwk, on_epoch=True, on_step=False)

        # --- Adição do RMSE em training_step ---
        self.train_rmse(predicted_labels.float(), true_labels.float())
        self.log("train_rmse", self.train_rmse, on_epoch=True, on_step=False)

        return loss  # A perda é retornada para o otimizador fazer o backpropagation

    def validation_step(self, batch, batch_idx):
        """
        Define o que acontece em cada passo de validação.
        Similar ao training_step, mas não calcula gradientes.
        """
        loss, true_labels, predicted_labels = self._shared_step(batch)

        # MODIFICADO AQUI
        self.valid_enem_acc(predicted_labels.float(), true_labels.float())
        self.log("valid_enem_acc", self.valid_enem_acc, on_epoch=True, on_step=False, prog_bar=True)

        self.log("valid_loss", loss, on_epoch=True, on_step=False)
        self.valid_mae(predicted_labels, true_labels)

        self.log("valid_mae", self.valid_mae,on_epoch=True, on_step=False, prog_bar=True) # prog_bar mostra no progresso da barra
        self.valid_qwk(predicted_labels, true_labels)

        self.log("valid_qwk", self.valid_qwk,on_epoch=True, on_step=False, prog_bar=True)

  # --- Adição do RMSE em validation_step ---
        self.valid_rmse(predicted_labels.float(), true_labels.float())
        self.log("valid_rmse", self.valid_rmse, on_epoch=True, on_step=False, prog_bar=True)

    def test_step(self, batch, batch_idx):
        """
        Define o que acontece em cada passo de teste (avaliação final).
        """
        # Não precisamos da perda para o teste, por isso o '_'
        _, true_labels, predicted_labels = self._shared_step(batch)

        self.test_enem_acc(predicted_labels.float(), true_labels.float())
        self.log("test_enem_acc", self.test_enem_acc, on_epoch=True, on_step=False)

        self.test_mae(predicted_labels, true_labels)
        self.log("test_mae", self.test_mae, on_epoch=True, on_step=False)

        self.test_qwk(predicted_labels, true_labels)
        self.log("test_qwk", self.test_qwk, on_epoch=True, on_step=False)

        self.test_rmse(predicted_labels.float(), true_labels.float())
        self.log("test_rmse", self.test_rmse, on_epoch=True, on_step=False)


    def configure_optimizers(self):
        """
        Configura o otimizador da rede neural.
        """
        # Otimizador Adam: otimizador popular e eficiente.
        # self.parameters() retorna todos os parâmetros treináveis do modelo.
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

**Classe MyDataset**


Este código define a classe MyDataset, que prepara os dados para o PyTorch. Ele armazena as features (X) e labels (y) em arrays NumPy. Os métodos **__len__** e **__getitem__** permitem que o PyTorch saiba o tamanho total do seu dataset e como acessar cada amostra individualmente (features e seu label correspondente) usando um índice. Isso é fundamental para organizar os dados para o treinamento do modelo.


In [ ]:
class MyDataset(Dataset): # A classe MyDataset herda de torch.utils.data.Dataset
    def __init__(self, feature_array, label_array, dtype=np.float32):
        """
        Método construtor da classe. É chamado quando você cria uma nova instância de MyDataset.

        Args:
            feature_array (np.ndarray): Um array NumPy contendo suas features (X).
                                        Por exemplo, X_train_std, X_val_std, X_test_std.
            label_array (np.ndarray): Um array NumPy contendo seus labels (y).
                                      Por exemplo, y_train, y_val, y_test.
            dtype (np.float32, optional): O tipo de dado em que as features serão convertidas.
                                          np.float32 é um tipo comum para entradas de redes neurais,
                                          pois economiza memória e é compatível com GPUs.
        """
        # Converte o array de features para o tipo de dado especificado.
        # Isso é importante para garantir que as features estejam no formato numérico
        # esperado pelo PyTorch (geralmente float32 ou float64).
        self.feature_data = feature_array.astype(dtype)

        # Armazena o array de labels.
        # Comentário: "Labels devem ser long para PyTorch" -- isso é uma dica importante.
        # Para problemas de classificação (e regressão ordinal como no presente trabalho, que usa classificadores binários internamente),
        # o PyTorch geralmente espera que os rótulos de classe sejam tensores do tipo Long (torch.long).
        # Se label_array for um NumPy array de inteiros, PyTorch lida com isso.
        self.labels = label_array

    def __getitem__(self, index):
        """
        Método mágico que permite acessar amostras do dataset usando índices, como em uma lista.
        Ex: dataset[0] chamaria __getitem__(0).

        Args:
            index (int): O índice da amostra que você deseja retornar.

        Returns:
            tuple: Uma tupla contendo (inputs, label) para a amostra no índice fornecido.
                   - inputs: As features (dados de entrada) da amostra.
                   - label: O rótulo (valor alvo) correspondente à amostra.
        """
        inputs = self.feature_data[index] # Pega a linha de features no 'index'
        label = self.labels[index]   # Pega o label correspondente no 'index'
        return inputs, label

    def __len__(self, ):
        """
        Método mágico que retorna o número total de amostras no dataset.
        Permite usar len(dataset).

        Returns:
            int: O número de linhas (amostras) no array de features.
        """
        return self.feature_data.shape[0] # Retorna o número de linhas (primeira dimensão) do array de features

**Classe CompetenceDataModule (Substitui DataModule)**


Esta é a parte central da adaptação para múltiplos modelos. O DataModule agora será específico para cada competência, recebendo os X e y já divididos e escalados.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

class CompetenceDataModule(pl.LightningDataModule): # A classe herda de pytorch_lightning.LightningDataModule
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test, batch_size, num_workers):
        """
        Método construtor da classe. É chamado quando é criado uma nova instância de CompetenceDataModule.

        Args:
            X_train (np.ndarray): Matriz de features para o conjunto de treinamento.
            y_train (np.ndarray): Vetor de labels (notas) para o conjunto de treinamento.
            X_val (np.ndarray): Matriz de features para o conjunto de validação.
            y_val (np.ndarray): Vetor de labels para o conjunto de validação.
            X_test (np.ndarray): Matriz de features para o conjunto de teste.
            y_test (np.ndarray): Vetor de labels para o conjunto de teste.
            batch_size (int): O número de amostras por lote (batch) que o modelo processará de cada vez.
            num_workers (int): O número de subprocessos a serem usados para carregamento de dados.
                                0 significa que o carregamento será feito no processo principal.
                                Valores > 0 podem acelerar o carregamento, mas podem causar problemas em ambientes como o Windows/WSL.
        """
        super().__init__() # Chama o construtor da classe base pl.LightningDataModule

        # Armazena os arrays NumPy dos conjuntos de dados
        # Esses dados já devem estar pré-processados e escalados (ex: X_train_std)
        # self.X_train = X_train
        # self.y_train = y_train
        # self.X_val = X_val
        # self.y_val = y_val
        # self.X_test = X_test
        # self.y_test = y_test

        self.X_train = torch.FloatTensor(X_train)
        self.y_train = torch.IntTensor(y_train)  # ou IntTensor dependendo do seu caso
        self.X_val = torch.FloatTensor(X_val)
        self.y_val = torch.IntTensor(y_val)
        self.X_test = torch.FloatTensor(X_test)
        self.y_test = torch.IntTensor(y_test)

        # Armazena configurações de DataLoader
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        """
        Este método é chamado pelo PyTorch Lightning para preparar os dados.
        Ele é chamado em cada "nó" (processo/GPU) em um ambiente distribuído.
        Normalmente, é aqui que fazemos a divisão de dados, escalamento, etc.
        Em nosso caso caso, ja fizemos isso no fluxo principal do script, então aqui ele
        apenas cria as instâncias de MyDataset.

        Args:
            stage (str, optional): Indica a fase atual ('fit', 'validate', 'test', 'predict').
                                   Permite lógica condicional se necessário. Ignorado aqui.
        """
        # Cria os objetos MyDataset para cada conjunto (treino, validação, teste).
        # MyDataset encapsula os arrays NumPy X e y em um formato que PyTorch pode usar.
        # self.train = MyDataset(self.X_train, self.y_train)
        # self.valid = MyDataset(self.X_val, self.y_val)
        # self.test = MyDataset(self.X_test, self.y_test)

        self.train = TensorDataset(self.X_train, self.y_train)
        self.valid = TensorDataset(self.X_val, self.y_val)
        self.test = TensorDataset(self.X_test, self.y_test)

    def train_dataloader(self):
        """
        Retorna o DataLoader para o conjunto de treinamento.
        """
        return DataLoader(self.train,
                          batch_size=self.batch_size, # Tamanho dos lotes para o treino
                          num_workers=self.num_workers, # Número de subprocessos para carregar dados
                          shuffle=True, # Embaralha os dados a cada epoch para evitar que o modelo "decore" a ordem
                          drop_last=True) # Se o último batch não tiver o tamanho completo, ele é descartado.
                                          # Útil para modelos que esperam batches de tamanho fixo, ou GPUs.

    def val_dataloader(self):
        """
        Retorna o DataLoader para o conjunto de validação.
        """
        return DataLoader(self.valid,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          drop_last=True) # Explicitly set drop_last to True for validation
                          # shuffle=False é o padrão para validação/teste, pois a ordem não importa.

    def test_dataloader(self):
        """
        Retorna o DataLoader para o conjunto de teste.
        """
        return DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers)
                          # shuffle=False é o padrão para validação/teste.

# Importação

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_train.csv")
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_val.csv")
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_test.csv")

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/df com métrica NILC/df_validation.csv")


In [ ]:
df_train['honore']

# Análise de divergência (test_hard and test_easy)

In [ ]:
# Definição do Tipo de Análise:
# Valores de 0 a 4: Competências do ENEM (C1 a C5)
# Valor 5: Nota final (soma das competências)
REFERENCE_CONCEPT = 4

In [ ]:
dataset = load_dataset("kamel-usp/aes_enem_dataset", "sourceAWithGraders", cache_dir="/tmp/aes_enem", trust_remote_code=True)

In [ ]:
def computa_diferenca(lists):
    """Identifica redações com grande divergência entre avaliadores (>80 pontos)"""
    if len(lists) < 3:
        return False

    referencia = lists[0][REFERENCE_CONCEPT]
    avaliador_a = lists[1][REFERENCE_CONCEPT]
    avaliador_b = lists[2][REFERENCE_CONCEPT]

    diff_ref_a = abs(referencia - avaliador_a)
    diff_ref_b = abs(referencia - avaliador_b)
    diff_a_b = abs(avaliador_a - avaliador_b)

    return diff_ref_a > 80 or diff_ref_b > 80 or diff_a_b > 80

In [ ]:
# Separação em conjuntos easy/hard
test_df = dataset["test"].to_pandas()

new_test_df = pd.merge(
    test_df.groupby(["id_prompt", "id"])
           .agg({"grades": list})
           .apply(lambda x: computa_diferenca(x['grades']), axis=1)
           .reset_index(),
    test_df,
    on=["id_prompt", "id"]
).rename(columns={0: "is_hard"})

In [ ]:
from datasets import Dataset

# O restante do seu código...
dataset["test_easy"] = Dataset.from_pandas(new_test_df[new_test_df["is_hard"]==False])
dataset["test_hard"] = Dataset.from_pandas(new_test_df[new_test_df["is_hard"]==True])

In [ ]:
# 1. Cria o DataFrame de Redações Fáceis (is_hard == False)
df_test_easy = new_test_df[new_test_df["is_hard"] == False].copy()

# 2. Cria o DataFrame de Redações Difíceis (is_hard == True)
df_test_hard = new_test_df[new_test_df["is_hard"] == True].copy()

# Exibe os tamanhos dos novos conjuntos para verificação
print(f"DataFrame EASY criado. Tamanho: {len(df_test_easy)}")
print(f"DataFrame HARD criado. Tamanho: {len(df_test_hard)}")

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/df features Murilo/test_murilo.csv")

In [ ]:

df_test['id'] = range(len(df_test))
df_test_easy['id'] = range(len(df_test_easy))
df_test_hard['id'] = range(len(df_test_hard))

In [ ]:
import pandas as pd

textos_easy = df_test_easy['id'].unique()
textos_hard = df_test_hard['id'].unique()

# 2. Criar os novos DataFrames (df_final_easy e df_final_hard) a partir do df_test principal
# Usamos o método .isin() para filtrar o df_test principal pelas chaves de texto.

# Filtrar as redações "fáceis"
df_final_easy = df_test[df_test['id'].isin(textos_easy)].copy()

# Filtrar as redações "difíceis"
df_final_hard = df_test[df_test['id'].isin(textos_hard)].copy()

# 3. Verificação dos tamanhos e da integridade dos dados
print("Separação concluída com o df_test como fonte:")
print(f"df_final_easy (Baixa Divergência): {len(df_final_easy)} registros")
print(f"df_final_hard (Alta Divergência): {len(df_final_hard)} registros")

# Verificação da soma
soma_total = len(df_final_easy) + len(df_final_hard)
print(f"Soma das partições: {soma_total} (Deve ser 209)")

In [ ]:
nome_do_arquivo_csv = 'df_test_easy.csv'
df_final_easy.to_csv(nome_do_arquivo_csv, index=False)

In [ ]:
nome_do_arquivo_csv = 'df_test_hard.csv'
df_final_hard.to_csv(nome_do_arquivo_csv, index=False)

In [ ]:
df_final_easy.head()

# TF-IDF

In [ ]:
# Aqui garantimos que lista de stop words do NLTK esteja disponível
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

In [ ]:
# Importação dos dataframes
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_train.csv")
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_val.csv")
df_test_easy = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_easy.csv")
df_test_hard = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_hard.csv")
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_test.csv")

In [ ]:
df_test_easy.shape

In [ ]:
df_train.shape

In [ ]:
df_val.shape

In [ ]:
df_test.shape

In [ ]:
# Pré-processmaento dos textos
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    stop_words_pt = set(stopwords.words('portuguese'))
    text = ' '.join([word for word in text.split() if word not in stop_words_pt])
    return text

In [ ]:
df_train['processed_text'] = df_train['essay_text'].apply(preprocess_text)
df_test['processed_text'] = df_test['essay_text'].apply(preprocess_text)
df_val['processed_text'] = df_val['essay_text'].apply(preprocess_text)
df_test_easy['processed_text'] = df_test_easy['essay_text'].apply(preprocess_text)


In [ ]:
# --- 1. Geração da Matriz TF-IDF Completa (apenas no treino) ---
# A matriz TF-IDF deve ser criada a partir do conjunto de treino para evitar vazamento
# de dados.
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words=stopwords.words('portuguese'))
X_train_tfidf_matrix = vectorizer.fit_transform(df_train['processed_text'])

In [ ]:
# Aplicação do pré-processamento nos textos
X_val_tfidf_matrix = vectorizer.transform(df_val['processed_text'])
X_test_tfidf_matrix = vectorizer.transform(df_test['processed_text'])
X_test_easy_tfidf_matrix = vectorizer.transform(df_test_easy['processed_text'])

In [ ]:
# --- 2. Seleção e União dos Índices de Features ---
# O número de features a selecionar por competência
K_FEATURES_PER_COMPETENCE = 1000

In [ ]:
# Lista para armazenar os índices das features selecionadas para cada competência
selected_features_indices_list = []

In [ ]:
for comp_idx in range(1, 6):
    print(f"Selecionando features para a Competência {comp_idx}...", flush=True)

    # Definir a variável alvo (y) para a competência atual no conjunto de treino
    y_train_comp = df_train[f'nota_competencia_{comp_idx}_ordinal']

    # Ajusta o SelectKBest para encontrar as K_FEATURES_PER_COMPETENCE mais relevantes
    selector = SelectKBest(f_classif, k=K_FEATURES_PER_COMPETENCE)
    selector.fit(X_train_tfidf_matrix, y_train_comp)

    # Adiciona os índices das features selecionadas à lista
    selected_features_indices_list.extend(selector.get_support(indices=True))

In [ ]:
# 3. Criar uma lista única de índices e ordenar
# O uso de np.unique remove duplicatas caso uma mesma feature seja relevante para
# mais de uma competência.
unique_selected_indices = np.sort(np.unique(selected_features_indices_list))

In [ ]:
# --- 4. Reduzir as Matrizes TF-IDF Originais ---
# Agora, aplicamos a lista de índices unificada a todas as matrizes TF-IDF.
X_train_tfidf_final = X_train_tfidf_matrix[:, unique_selected_indices]
X_val_tfidf_final = X_val_tfidf_matrix[:, unique_selected_indices]
X_test_tfidf_final = X_test_tfidf_matrix[:, unique_selected_indices]
X_test_easy_tfidf_final = X_test_easy_tfidf_matrix[:, unique_selected_indices]

In [ ]:
# Converte para arrays densos, se necessário, para concatenação futura
X_train_tfidf_final_array = X_train_tfidf_final.toarray()
X_val_tfidf_final_array = X_val_tfidf_final.toarray()
X_test_tfidf_final_array = X_test_tfidf_final.toarray()
X_test_easy_tfidf_final_array = X_test_easy_tfidf_final.toarray()

In [ ]:
print(f"Matriz TF-IDF final do treino. Dimensão: {X_train_tfidf_final_array.shape}")
print(f"Matriz TF-IDF final do teste. Dimensão: {X_test_tfidf_final_array.shape}")
print(f"Matriz TF-IDF final da validação. Dimensão: {X_val_tfidf_final_array.shape}")
print(f"Matriz TF-IDF final da test easy. Dimensão: {X_test_easy_tfidf_final_array.shape}")


Treinamento base test inteira

In [ ]:
# --- 4. Loop para Treinar um Modelo para Cada Competência ---
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6): # Para competência 1 a 5
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    # Concatena as features artesanais com as features TF-IDF
    X_train =  X_train_tfidf_final_array
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val =  X_val_tfidf_final_array
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_tfidf_final_array
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    # Escalamento de Features (Fit no treino, transform nos outros)
    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)


    # Criar DataModule para a competência atual
    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()


    # Inicializar e treinar o modelo PyTorch Lightning
    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    # Callbacks e Logger
    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    # Treinar
    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    # Avaliar no conjunto de teste (avaliação final)
    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    # Geração do classification Report
    # Obter o modelo treinado
    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    # Fazer previsões no conjunto de teste para obter o y_pred
    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test # o y_test já foi definido no loop

    # Discretizar as previsões para o formato de classe
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    # Gerar os relatórios de classificação
    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

Treinamento com Easy test

In [ ]:
# --- 4. Loop para Treinar um Modelo para Cada Competência ---
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6): # Para competência 1 a 5
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    # Concatena as features artesanais com as features TF-IDF
    X_train =  X_train_tfidf_final_array
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val =  X_val_tfidf_final_array
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_easy_tfidf_final_array
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    # Escalamento de Features (Fit no treino, transform nos outros)
    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)


    # Criar DataModule para a competência atual
    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()


    # Inicializar e treinar o modelo PyTorch Lightning
    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    # Callbacks e Logger
    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    # Treinar
    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    # Avaliar no conjunto de teste (avaliação final)
    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    # Geração do classification Report
    # Obter o modelo treinado
    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    # Fazer previsões no conjunto de teste para obter o y_pred
    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test # o y_test já foi definido no loop

    # Discretizar as previsões para o formato de classe
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    # Gerar os relatórios de classificação
    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# NILC

In [ ]:
# Importação dos dataframes
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_train.csv", decimal=',')
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_val.csv", decimal=',')
df_test_easy = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_easy.csv", decimal=',')
df_test_hard = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_hard.csv", decimal=',')
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_test.csv", decimal=',')

In [ ]:

import numpy as np
import pandas as pd

def prepare_feature_matrix_pure_handcrafted(df):
    """
    Prepara a matriz de features usando apenas as features artesanais.
    """
    print("\nPreparando a matriz de features (X) com Handcrafted Puro...", flush=True)

    # 1. Pré-processar Features Hand-crafted
    primeira_coluna_metrica = ' adjective_ratio'
    ultima_coluna_metrica = 'ratio_function_to_content_words'
    colunas_handcrafted_nomes = df.loc[:, primeira_coluna_metrica:ultima_coluna_metrica].columns.tolist()

    for col in colunas_handcrafted_nomes:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    handcrafted_features_matrix = df[colunas_handcrafted_nomes].fillna(df[colunas_handcrafted_nomes].mean()).values
    X_pure_handcrafted = handcrafted_features_matrix
    print(f"Matriz de features X_pure_handcrafted pronta. Dimensão: {X_pure_handcrafted.shape}")

    return X_pure_handcrafted

X_train_handcrafted = prepare_feature_matrix_pure_handcrafted(df_train)
X_val_handcrafted = prepare_feature_matrix_pure_handcrafted(df_val)
X_test_handcrafted = prepare_feature_matrix_pure_handcrafted(df_test)
X_test_easy_handcrafted = prepare_feature_matrix_pure_handcrafted(df_test_easy)

Treinamento com a base teste inteira

In [ ]:

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = X_train_handcrafted
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = X_val_handcrafted
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_handcrafted
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)


    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)


    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()


    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    # Treinar
    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

Treinamento com easy test

In [ ]:

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = X_train_handcrafted
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = X_val_handcrafted
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_easy_handcrafted
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)


    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)


    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()


    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    # Treinar
    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# BERT

In [ ]:
# Importação dos dataframes
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_train.csv", decimal=',')
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_val.csv", decimal=',')
df_test_easy = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_easy.csv", decimal=',')
df_test_hard = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_hard.csv", decimal=',')
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_test.csv", decimal=',')

def prepare_feature_matrix_pure_bert(df):
    """
    Prepara a matriz de features usando apenas os embeddings do BERT.
    """
    print("\nPreparando a matriz de features (X) com BERT Puro...", flush=True)

    # 1. Pré-processar BERT Embeddings (esta etapa permanece a mesma)
    df['bert_embedding'] = df['bert_embedding'].apply(
        lambda x: np.array(ast.literal_eval(re.sub(r'\s+', ',', str(x).strip('[]')).strip(',')))
        if pd.notna(x) and isinstance(x, str) and x.strip() else np.zeros(768)
    )
    bert_embeddings_matrix = np.stack(df['bert_embedding'].values)

    # 3. Retorne apenas a matriz de embeddings do BERT
    X_pure_bert = bert_embeddings_matrix
    print(f"Matriz de features X_pure_bert pronta. Dimensão: {X_pure_bert.shape}")

    return X_pure_bert

# Passo 2: Aplicar a função a cada DataFrame para obter as matrizes X
# Os nomes das variáveis foram alterados para refletir a abordagem pura
X_train_pure = prepare_feature_matrix_pure_bert(df_train)
X_val_pure = prepare_feature_matrix_pure_bert(df_val)
X_test_pure = prepare_feature_matrix_pure_bert(df_test)
X_test_easy_pure = prepare_feature_matrix_pure_bert(df_test_easy)

Treinamento com toda base de teste

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = X_train_pure
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = X_val_pure
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_pure
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )


    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

Treinamento apenas com easy test

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = X_train_pure
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = X_val_pure
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_easy_pure
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )


    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# Murilo

In [ ]:
df_murilo_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/df features Murilo/train_murilo.csv")
df_murilo_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/df features Murilo/validation_murilo.csv")
df_murilo_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/df features Murilo/test_murilo.csv")
df_murilo_test_easy = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/df features Murilo/df_murilo_test_easy.csv")

In [ ]:
nota_total_col = 'nota_total'
competencia_cols = ['nota_c1', 'nota_c2', 'nota_c3', 'nota_c4', 'nota_c5']

In [ ]:
def mapeamento_ordinal(df):
    # Dicionário que mapeia a nota real para o valor ordinal
    mapeamento_competencia_ordinal = {
        0: 0,
        40: 1,
        80: 2,
        120: 3,
        160: 4,
        200: 5
    }

    for col_name in competencia_cols:
        df[f'{col_name}_ordinal'] = df[col_name].map(mapeamento_competencia_ordinal)
    return df

df_test = mapeamento_ordinal(df_murilo_test)
df_train = mapeamento_ordinal(df_murilo_train)
df_val = mapeamento_ordinal(df_murilo_val)
df_test_easy = mapeamento_ordinal(df_murilo_test_easy)

In [ ]:
def prepare_feature_matrix_pure_handcrafted(df):
    """
    Prepara a matriz de features usando apenas as features artesanais.
    """

    primeira_coluna_metrica = 'n_verbos_e_pronomes_1ps'
    ultima_coluna_metrica = 'similaridade_tit_1_constituicao_bertimbau'
    colunas_handcrafted_nomes = df.loc[:, primeira_coluna_metrica:ultima_coluna_metrica].columns.tolist()

    for col in colunas_handcrafted_nomes:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    handcrafted_features_matrix = df[colunas_handcrafted_nomes].fillna(df[colunas_handcrafted_nomes].mean()).values

    X_pure_handcrafted = handcrafted_features_matrix
    # print(f"Matriz de features X_pure_handcrafted pronta. Dimensão: {X_pure_handcrafted.shape}")

    return X_pure_handcrafted

# Passo 2: Aplicar a função a cada DataFrame para obter as matrizes X
X_train_handcrafted_Murilo = prepare_feature_matrix_pure_handcrafted(df_train)
X_val_handcrafted_Murilo = prepare_feature_matrix_pure_handcrafted(df_val)
X_test_handcrafted_Murilo = prepare_feature_matrix_pure_handcrafted(df_test)
X_test_easy_handcrafted_Murilo = prepare_feature_matrix_pure_handcrafted(df_test_easy)

treinamento base test inteira

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)
    y_train_series = df_train[f'nota_c{comp_idx}_ordinal']
    y_val_series = df_val[f'nota_c{comp_idx}_ordinal']
    y_test_series = df_test[f'nota_c{comp_idx}_ordinal']

    X_train = X_train_handcrafted_Murilo
    y_train = df_train[f'nota_c{comp_idx}_ordinal'].values
    y_train = y_train_series.fillna(0).astype(int).values

    X_val = X_val_handcrafted_Murilo
    y_val = df_val[f'nota_c{comp_idx}_ordinal'].values
    y_val = y_val_series.fillna(0).astype(int).values

    X_test = X_test_handcrafted_Murilo
    y_test = df_test[f'nota_c{comp_idx}_ordinal'].values
    y_test = y_test_series.fillna(0).astype(int).values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

Treinamento easy test

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)
    y_train_series = df_train[f'nota_c{comp_idx}_ordinal']
    y_val_series = df_val[f'nota_c{comp_idx}_ordinal']
    y_test_series = df_test_easy[f'nota_c{comp_idx}_ordinal']

    X_train = X_train_handcrafted_Murilo
    y_train = df_train[f'nota_c{comp_idx}_ordinal'].values
    y_train = y_train_series.fillna(0).astype(int).values

    X_val = X_val_handcrafted_Murilo
    y_val = df_val[f'nota_c{comp_idx}_ordinal'].values
    y_val = y_val_series.fillna(0).astype(int).values

    X_test = X_test_easy_handcrafted_Murilo
    y_test = df_test_easy[f'nota_c{comp_idx}_ordinal'].values
    y_test = y_test_series.fillna(0).astype(int).values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# TF-IDF + NILC  

In [ ]:
# Importação dos dataframes
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_train.csv")
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_val.csv")
df_test_easy = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_easy.csv")
df_test_hard = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_hard.csv")
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_test.csv")

def prepare_feature_matrix_pure_handcrafted(df):

    primeira_coluna_metrica = ' adjective_ratio'
    ultima_coluna_metrica = 'ratio_function_to_content_words'
    colunas_handcrafted_nomes = df.loc[:, primeira_coluna_metrica:ultima_coluna_metrica].columns.tolist()

    for col in colunas_handcrafted_nomes:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    handcrafted_features_matrix = df[colunas_handcrafted_nomes].fillna(df[colunas_handcrafted_nomes].mean()).values

    X_pure_handcrafted = handcrafted_features_matrix
    print(f"Matriz de features X_pure_handcrafted pronta. Dimensão: {X_pure_handcrafted.shape}")

    return X_pure_handcrafted

X_train_handcrafted = prepare_feature_matrix_pure_handcrafted(df_train)
X_val_handcrafted = prepare_feature_matrix_pure_handcrafted(df_val)
X_test_handcrafted = prepare_feature_matrix_pure_handcrafted(df_test)
X_test_easy_handcrafted = prepare_feature_matrix_pure_handcrafted(df_test_easy)

Treinamento base teste inteira

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_handcrafted, X_test_tfidf_final_array))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

Treinamento teste easy

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_handcrafted, X_test_easy_tfidf_final_array))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# TF-IDF + BERT

In [ ]:
# # Importação dos dataframes
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_val.csv")
# df_test_easy = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_easy.csv")
# df_test_hard = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_hard.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_test.csv")

# def prepare_feature_matrix_pure_bert(df):
#     df['bert_embedding'] = df['bert_embedding'].apply(
#         lambda x: np.array(ast.literal_eval(re.sub(r'\s+', ',', str(x).strip('[]')).strip(',')))
#         if pd.notna(x) and isinstance(x, str) and x.strip() else np.zeros(768)
#     )
#     bert_embeddings_matrix = np.stack(df['bert_embedding'].values)

#     X_pure_bert = bert_embeddings_matrix
#     print(f"Matriz de features X_pure_bert pronta. Dimensão: {X_pure_bert.shape}")

#     return X_pure_bert
# X_train_pure = prepare_feature_matrix_pure_bert(df_train)
# X_val_pure = prepare_feature_matrix_pure_bert(df_val)
# X_test_pure = prepare_feature_matrix_pure_bert(df_test)

Treinamento com a base teste inteira

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_pure, X_test_tfidf_final_array))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

Treinamento com teste easy

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_pure, X_test_easy_tfidf_final_array))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))
    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# TF-IDF + Murilo

Treinamento base test inteira

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")


resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted_Murilo, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted_Murilo, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_handcrafted_Murilo, X_test_tfidf_final_array))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

treinamento easy test

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")


resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted_Murilo, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted_Murilo, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_handcrafted_Murilo, X_test_easy_tfidf_final_array))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# NILC + BERT

In [ ]:
# Importação dos dataframes
df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_train.csv")
df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_val.csv")
df_test_easy = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_easy.csv")
df_test_hard = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/test easy and hard/df_test_hard.csv")
df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/Base correta/NILC + Embedding + ORD (final df)/df_test.csv")

def prepare_feature_matrix(df):
    df['bert_embedding'] = df['bert_embedding'].apply(
        lambda x: np.array(ast.literal_eval(re.sub(r'\s+', ',', str(x).strip('[]')).strip(',')))
        if pd.notna(x) and isinstance(x, str) and x.strip() else np.zeros(768)
    )
    bert_embeddings_matrix = np.stack(df['bert_embedding'].values)

    primeira_coluna_metrica = ' adjective_ratio'
    ultima_coluna_metrica = 'ratio_function_to_content_words'
    colunas_handcrafted_nomes = df.loc[:, primeira_coluna_metrica:ultima_coluna_metrica].columns.tolist()

    for col in colunas_handcrafted_nomes:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    handcrafted_features_matrix = df[colunas_handcrafted_nomes].fillna(df[colunas_handcrafted_nomes].mean()).values

    X_combined = np.hstack((bert_embeddings_matrix, handcrafted_features_matrix))
    print(f"Matriz de features X_combined pronta. Dimensão: {X_combined.shape}")

    return X_combined

X_train_combined = prepare_feature_matrix(df_train)
X_val_combined = prepare_feature_matrix(df_val)
X_test_combined = prepare_feature_matrix(df_test)
X_test_easy_combined = prepare_feature_matrix(df_test_easy)

treinamento base teste inteira

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu" #

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = X_train_combined
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = X_val_combined
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_combined
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

treinamento base easy

In [ ]:
resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu" #

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = X_train_combined
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = X_val_combined
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = X_test_easy_combined
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# NILC + Murilo

treinamento base test inteira

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_handcrafted, X_test_handcrafted_Murilo))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))
    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

treinamento easy test

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_handcrafted, X_test_easy_handcrafted_Murilo))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# BERT + Murilo

treinamento base test inteira

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_pure, X_test_handcrafted_Murilo))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )


    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )


    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)


    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]
    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

In [ ]:
#treinamento easy test

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_pure, X_test_easy_handcrafted_Murilo))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )


    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )


    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)


    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]
    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# TF-IDF + NILC + BERT

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_combined, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_combined, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_combined, X_test_tfidf_final_array))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)
    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )


    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

treinamento base easy

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_combined, X_train_tfidf_final_array))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_combined, X_val_tfidf_final_array))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_combined, X_test_easy_tfidf_final_array))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)
    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )


    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# TF IDF + NILC + Murilo

treinamento base test inteira

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_tfidf_final_array, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_tfidf_final_array, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_handcrafted, X_test_tfidf_final_array, X_test_handcrafted_Murilo))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)


    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

treinamento base test easy

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_tfidf_final_array, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_tfidf_final_array, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_handcrafted, X_test_easy_tfidf_final_array, X_test_easy_handcrafted_Murilo))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)


    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# TF IDF + BERT + Murilo

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_tfidf_final_array, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_tfidf_final_array, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_pure, X_test_tfidf_final_array, X_test_handcrafted_Murilo))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

teste easy

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_tfidf_final_array, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_tfidf_final_array, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_pure, X_test_easy_tfidf_final_array, X_test_easy_handcrafted_Murilo))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# NILC + BERT + Murilo


In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_handcrafted, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_handcrafted, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_pure, X_test_handcrafted, X_test_handcrafted_Murilo))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

test easy

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_pure, X_train_handcrafted, X_train_handcrafted_Murilo))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_pure, X_val_handcrafted, X_val_handcrafted_Murilo))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_pure, X_test_easy_handcrafted, X_test_easy_handcrafted_Murilo))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test
    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

# TF IDF + NILC + BERT + Murilo


In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_tfidf_final_array, X_train_handcrafted_Murilo, X_train_pure))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_tfidf_final_array, X_val_handcrafted_Murilo, X_val_pure))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_handcrafted, X_test_tfidf_final_array, X_test_handcrafted_Murilo, X_test_pure))
    y_test = df_test[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))
    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)

teste easy

In [ ]:
# df_train = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_validation.csv")
# df_test = pd.read_csv("/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame/Df text + metrics + embeddings + vet_ordinal/df_test.csv")

resultados_por_competencia = {}
device_type = "cuda" if torch.cuda.is_available() else "cpu"

for comp_idx in range(1, 6):
    print(f"\n--- Treinando Modelo para Competência {comp_idx} ---", flush=True)

    X_train = np.hstack((X_train_handcrafted, X_train_tfidf_final_array, X_train_handcrafted_Murilo, X_train_pure))
    y_train = df_train[f'nota_competencia_{comp_idx}_ordinal'].values

    X_val = np.hstack((X_val_handcrafted, X_val_tfidf_final_array, X_val_handcrafted_Murilo, X_val_pure))
    y_val = df_val[f'nota_competencia_{comp_idx}_ordinal'].values

    X_test = np.hstack((X_test_easy_handcrafted, X_test_easy_tfidf_final_array, X_test_easy_handcrafted_Murilo, X_test_easy_pure))
    y_test = df_test_easy[f'nota_competencia_{comp_idx}_ordinal'].values

    print(f"Divisão de dados para Competência {comp_idx}: Treino={X_train.shape}, Validação={X_val.shape}, Teste={X_test.shape}", flush=True)

    scaler_comp = StandardScaler()
    X_train_std = scaler_comp.fit_transform(X_train)
    X_val_std = scaler_comp.transform(X_val)
    X_test_std = scaler_comp.transform(X_test)
    print(f"Features escaladas para Competência {comp_idx}.", flush=True)

    data_module_competencia = CompetenceDataModule(X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, BATCH_SIZE, NUM_WORKERS)
    data_module_competencia.setup()

    input_dim = X_train_std.shape[1]

    pytorch_model_competencia = MultiLayerPerceptron(
        input_size=input_dim,
        hidden_units=(256, 128, 64),
        num_classes=NUM_CLASSES_COMPETENCIA
    )
    lightning_model_competencia = LightningMLP(
        model=pytorch_model_competencia,
        learning_rate=LEARNING_RATE
    )

    model_checkpoint_callback = ModelCheckpoint(
        save_top_k=1, mode="min", monitor="valid_mae",
        dirpath=f"logs_comp_{comp_idx}/", filename=f"best_model_comp_{comp_idx}"
    )
    logger_competencia = CSVLogger(save_dir="logs/", name=f"mlp-coral-comp-{comp_idx}")


    early_stop_callback = EarlyStopping(
    monitor="valid_qwk",
    min_delta=0.00,
    patience=50,
    verbose=True,
    mode="max"
    )

    callbacks_list = [
        model_checkpoint_callback,
        early_stop_callback
    ]

    trainer_competencia = pl.Trainer(
        max_epochs=NUM_EPOCHS,
        callbacks=callbacks_list,
        accelerator=device_type,
        devices=1,
        logger=logger_competencia,
        deterministic=True,
        log_every_n_steps=10
    )

    start_time = time.time()
    print(f"Treinando modelo para Competência {comp_idx}...", flush=True)
    trainer_competencia.fit(model=lightning_model_competencia, datamodule=data_module_competencia)
    runtime = (time.time() - start_time)/60
    print(f"Treinamento para Competência {comp_idx} levou {runtime:.2f} min.", flush=True)

    print(f"Avaliando modelo para Competência {comp_idx} no conjunto de teste...", flush=True)
    test_results = trainer_competencia.test(model=lightning_model_competencia, datamodule=data_module_competencia)
    resultados_por_competencia[f'competencia_{comp_idx}'] = test_results[0]

    model_path = model_checkpoint_callback.best_model_path
    best_model = LightningMLP.load_from_checkpoint(model_path, model=pytorch_model_competencia, learning_rate=LEARNING_RATE)
    best_model.eval()

    y_pred_list = []
    with torch.no_grad():
        for batch in data_module_competencia.test_dataloader():
            features, _ = batch
            logits = best_model(features)
            probas = torch.sigmoid(logits)
            predicted_labels = proba_to_label(probas)
            y_pred_list.append(predicted_labels.cpu().numpy())

    y_pred = np.hstack(y_pred_list)
    y_true = y_test

    y_pred_discreto = np.clip(np.round(y_pred), 0, NUM_CLASSES_COMPETENCIA - 1).astype(int)

    print("--- Relatório de Classificação da Competência ", comp_idx, " ---")
    print(classification_report(y_true, y_pred_discreto, zero_division=0.0))
    print("\n--- Matriz de Confusão da Competência ", comp_idx, " ---")
    print(confusion_matrix(y_true, y_pred_discreto))

    print(f"--- Modelo para Competência {comp_idx} concluído ---", flush=True)

print("\n--- Todos os modelos de competência foram treinados e avaliados ---", flush=True)
for comp, results in resultados_por_competencia.items():
    print(f"{comp}: QWK = {results['test_qwk']:.4f}, RMSE = {results['test_rmse']:.4f}, HDIV = {1- results['test_enem_acc']:.4f}", flush=True)